Zachary Proom

EN.605.646.81: Natural Language Processing

# Lab #4

In [5]:
## a

In [6]:
import re

# Define the tokenizer function.
def tokenize_text(input):
    """        
    Parameters
    ----------
    input : str
        The input text that gets tokenized.
    
    Returns
    -------
    An ordered list of normalized tokens.
    """
    
    # Add whitespace around punctuation. Exclude periods and hyphens for now
    # because they require extra care.
    input_nopunct = re.sub("(\.\.\.)", r" \1 ", input) # Ellipsis
    input_nopunct = re.sub("([^\w\d\.-])", r" \1 ", input_nopunct)

    # Add whitespace around a period only when it's at the end of a sentence.
    # To detect this, check if the following conditions are true:
    #     (1) There is whitespace after the period.
    #     (2) The period is at the end of a line.
    #     (3) There is a lowercase letter before the period, and an uppercase
    #         letter after the period.
    # In cases (1) and (2), the period cannot be preceded by a period because
    # it would be part of an ellipsis.
    input_nopunct = re.sub("(?<![\.])(\.)\s", r" \1 ", input_nopunct)
    input_nopunct = re.sub("(?<![\.])(\.)$", r" \1 ", input_nopunct)
    input_nopunct = re.sub("([a-z])(\.)([A-Z])", r"\1 \2 \3", input_nopunct)

    # Add whitespace around a hyphen except when it's used between letters 
    # (e.g. so-called).
    input_nopunct = re.sub(r'(?<![a-zA-Z])-(?![a-zA-Z])', ' - ', input_nopunct)
    
    # Lower-case all the words in the input text.
    input_lowercase = input_nopunct.lower()
    
    # Split all the words by whitespace.
    input_wssplit = input_lowercase.split()
    
    return input_wssplit

In [7]:
# Test the tokenizer function on the example sentence in the prompt.
tokenize_text("NAC has developed a National HIV/AIDS/STI/TB Intervention Strategic Plan (2002-2005) that aims to reduce the HIV prevalence rate among Zambians from 19.3% to 11.7% and improve the health status of people living with HIV/AIDS by 2005.")

ERROR! Session/line number was not unique in database. History logging moved to new session 6


['nac',
 'has',
 'developed',
 'a',
 'national',
 'hiv',
 '/',
 'aids',
 '/',
 'sti',
 '/',
 'tb',
 'intervention',
 'strategic',
 'plan',
 '(',
 '2002',
 '-',
 '2005',
 ')',
 'that',
 'aims',
 'to',
 'reduce',
 'the',
 'hiv',
 'prevalence',
 'rate',
 'among',
 'zambians',
 'from',
 '19.3',
 '%',
 'to',
 '11.7',
 '%',
 'and',
 'improve',
 'the',
 'health',
 'status',
 'of',
 'people',
 'living',
 'with',
 'hiv',
 '/',
 'aids',
 'by',
 '2005',
 '.']

I define a function above called tokenize_text(). It uses the re module to pad punctuation characters with spaces, so the string can be split on spaces. Before being split, the string is also converted to lowercase. The function adds whitespace around all punctuation, and it handles periods and hyphens with extra care. In particular, the function tries to only add whitespace around periods when it's at the end of a sentence. It also only tries to add whitespace around hyphens when they aren't between letters.

As you can see above, the function works on the test sentence. In my first attempt, I did not think about splitting around ellipses. I noticed my function didn't split ellipses on the first ten lines in tokens.txt, and I added more code to my function to handle them. I also didn't think about typos where there's no space between the last letter of a sentence and the first letter of the proceeding sentence (e.g. "ham.I do"). I had to add condition (3) in my function to deal with this correctly. My function still has trouble handling some less common initialisms like "U.S. of A". I'm unsure whether my tokenizer function should split contractions like "they're". Currently, it splits on the apostrophe.

Below I show the result of processing the first ten lines of tokens.txt.

In [8]:
with open("tokens.txt") as file:
    for i in range(10):
        line = next(file).strip()
        print(tokenize_text(line))

['russian', 'for', 'plastic', 'bag', 'is', 'полиэтиленовый', 'пакет', '.', '7.3', 'out', 'of', '10', 'statistics', 'is', 'made', 'up', '.', 'i', 'do', 'not', 'like', 'green', 'eggs', 'and', 'ham', '.', 'i', 'do']
['not', 'like', 'them', 'sam-i-am', '.', 'dr', '.', 'mulholland', 'lives', 'on', 'mulholland', 'dr', '.', 'in', 'hollywood', '.', '1', ',', '2', ',', '3', '...', 'slashdot.com', 'has', 'some', 'interesting']
['articles', '.', 'i', "'", 'm', 'going', 'to', 'update', 'my', 'resumé', '.', 'j.h.u', '.', 'has', 'a', 'great', 'la-crosse', 'team', '.', 'born', 'in', 'the', 'u.s', '.', 'of', 'a', '.', 'incorrect', 'plurala', 'can', 'be']
['fun', '.', 'is', 'capitalization', '(', 'sp', '?', ')', 'truly', 'necessary', '?', 'i', 'think', 'lower', 'case', 'is', 'more', 'legible', '.', 'when', 'people', 'write', 'in', 'all', 'caps', ',', 'it', 'feels']
['like', 'they', "'", 're', 'yelling', '!', 'it', 'is', 'precisely', 'to', 'these', 'substances', 'that', 'the', 'so-called', 'french', 'pa